# Regras de associação

In [2]:
%pip install apyori pandas &> /dev/null


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from apyori import apriori


In [4]:
base= pd.read_csv('./data.csv', sep=';', encoding='utf-8').map(lambda bool_str: bool_str == "Sim")
base


,Leite,Café,Cerveja,Pão,Manteiga,Arroz,Feijão
0,False,True,False,True,True,False,False
1,True,False,True,True,True,False,False
2,False,True,False,True,True,False,False
3,True,True,False,True,True,False,False
4,False,False,True,False,False,False,False
5,False,False,False,False,True,False,False
6,False,False,False,True,False,False,False
7,False,False,False,False,False,False,True
8,False,False,False,False,False,True,True
9,False,False,False,False,False,True,False


In [5]:
base.shape


(10, 7)

Transformar o dataframe em uma lista de transações.

In [10]:
txs = []
for i in range(len(base)):
  l = []
  cols_len = len(base.columns)
  for j, col_name in enumerate(base.columns):
    cell_val = base.values[i, j]
    if cell_val:
      l.append(col_name)
    else:
      l.append(f"Não {col_name}")
  txs.append(l)

for el in txs[:2]: print(el)
print("...")


['Não Leite', 'Café', 'Não Cerveja', 'Pão', 'Manteiga', 'Não Arroz', 'Não Feijão']
['Leite', 'Não Café', 'Cerveja', 'Pão', 'Manteiga', 'Não Arroz', 'Não Feijão']
...


Executando o algoritmo apriori e salvando as regras obtidas.

In [11]:
MIN_SUPPORT = 0.3
MIN_CONFIDENCE = 0.8

rules = apriori(txs, min_support=MIN_SUPPORT, min_confidence=MIN_CONFIDENCE)
rules_list = list(rules)

for rule in rules_list: print(rule)


RelationRecord(items=frozenset({'Não Arroz'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não Arroz'}), confidence=0.8, lift=1.0)])
RelationRecord(items=frozenset({'Não Cerveja'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não Cerveja'}), confidence=0.8, lift=1.0)])
RelationRecord(items=frozenset({'Não Feijão'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não Feijão'}), confidence=0.8, lift=1.0)])
RelationRecord(items=frozenset({'Não Leite'}), support=0.8, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Não Leite'}), confidence=0.8, lift=1.0)])
RelationRecord(items=frozenset({'Café', 'Manteiga'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Café'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)])
RelationRecord(items=frozenset({'Café', 'Não Arroz'}), 

Ref:
- https://www.section.io/engineering-education/apriori-algorithm-in-python/
- https://splunktool.com/understanding-apyoris-output

In [12]:
Antecedente = []
Consequente = []
support = []
confidence = []
lift = []

for resultado in rules_list:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    support.append(s)
    confidence.append(c)
    lift.append(l)
    final_df = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': support, 'Confidence': confidence, 'lift': lift})


In [13]:
final_df


,Antecedente,Consequente,suporte,Confidence,lift
0,[Café],[Manteiga],0.3,1.0,2.000000
1,[Café],[Não Arroz],0.3,1.0,1.250000
2,[Café],[Não Cerveja],0.3,1.0,1.250000
3,[Café],[Não Feijão],0.3,1.0,1.250000
4,[Café],[Pão],0.3,1.0,2.000000
...,...,...,...,...,...
440,"[Manteiga, Não Cerveja, Não Arroz, Café, Pão]",[Não Feijão],0.3,1.0,1.250000
441,"[Manteiga, Não Arroz, Não Feijão, Café, Pão]",[Não Cerveja],0.3,1.0,1.250000
442,"[Manteiga, Não Cerveja, Não Feijão, Café, Pão]",[Não Arroz],0.3,1.0,1.250000
443,"[Não Cerveja, Não Arroz, Não Feijão, Café, Pão]",[Manteiga],0.3,1.0,2.000000


**Podemos ordenar estas regras por uma métrica desejada**

In [14]:
final_df.sort_values(by='lift', ascending=False)


,Antecedente,Consequente,suporte,Confidence,lift
444,"[Manteiga, Não Cerveja, Não Arroz, Não Feijão,...",[Café],0.3,1.0,3.333333
426,"[Não Cerveja, Pão, Manteiga]","[Não Feijão, Café, Não Arroz]",0.3,1.0,3.333333
361,"[Não Feijão, Café]","[Não Cerveja, Pão, Manteiga]",0.3,1.0,3.333333
369,"[Não Cerveja, Pão, Manteiga]","[Não Feijão, Café]",0.3,1.0,3.333333
358,[Café],"[Não Cerveja, Não Feijão, Pão, Manteiga]",0.3,1.0,3.333333
...,...,...,...,...,...
102,"[Não Café, Não Manteiga]",[Não Cerveja],0.4,0.8,1.000000
106,"[Não Café, Não Pão]",[Não Cerveja],0.4,0.8,1.000000
108,"[Não Café, Não Feijão]",[Não Leite],0.4,0.8,1.000000
20,[Pão],[Não Cerveja],0.4,0.8,1.000000
